In [1]:
using CSV
using Distances
using Random

In [2]:
Random.seed!(4)
n = 20      # The number of patients
c_2 = 5     # unit distance cost
patients = [2:n+1;]
V = [1:n+1;]  # 1 denotes depot.    
q = [rand(12:15,n+1);]   # Requirment of each patient
arcs = []
for i in V
    for j in V 
        push!(arcs,(i,j))
    end
end
Q = 100    # Capacity
D = 10  # Max flying distance
q

21-element Array{Int64,1}:
 15
 15
 12
 14
 14
 13
 12
 15
 15
 12
 15
 15
 14
 12
 13
 14
 15
 15
 15
 15
 14

In [3]:
data = CSV.read("Patients21214_TableToExcel.csv")

,OID,CID,POINT_X,POINT_Y
,Int64,Int64,Float64,Float64
1,0,0,-76.5617,39.3524
2,1,1,-76.5695,39.3364
3,2,2,-76.5613,39.3383
4,3,3,-76.5504,39.3508
5,4,4,-76.5682,39.362
6,5,5,-76.5583,39.3494
7,6,6,-76.579,39.3474
8,7,7,-76.5558,39.3603
9,8,8,-76.563,39.3652


In [4]:
x_loc = data[:,3];
y_loc = data[:,4];
d = zeros(n+1,n+1)   # Distance matrix
for i = 1: length(V)
    for j = 1: length(V)
        d[i,j] = haversine((x_loc[i],y_loc[i]),(x_loc[j],y_loc[j]),6372.8) 
    end
end
d

21×21 Array{Float64,2}:
 0.0       1.90638   1.5708    0.988645  …  0.446169  0.740393  1.12463 
 1.90638   0.0       0.732939  2.29834      1.46164   1.31052   0.823554
 1.5708    0.732939  0.0       1.68166      1.15375   1.24407   0.855072
 0.988645  2.29834   1.68166   0.0          1.12801   1.56688   1.73702 
 1.20083   2.84838   2.69785   1.97172      1.55116   1.54      2.02834 
 0.442061  1.73457   1.25928   0.705256  …  0.422781  0.862436  1.06606 
 1.59118   1.47947   1.83039   2.49281      1.37013   0.929567  1.00538 
 1.01226   2.90987   2.49334   1.15373      1.45004   1.72712   2.13684 
 1.42326   3.25164   2.99273   1.93015      1.84647   1.95038   2.42982 
 0.33765   1.76791   1.56459   1.31296      0.431385  0.487566  0.94869 
 1.286     1.04709   1.29899   2.07754   …  0.95699   0.546763  0.469078
 0.373232  2.27646   1.93561   1.04387      0.81887   1.06162   1.48362 
 0.899612  1.5108    1.5601    1.82905      0.73985   0.347045  0.736389
 0.881433  2.09172   2.0593

In [5]:
using JuMP, Gurobi 
v = 1:3  # # of drones
m = Model(Gurobi.Optimizer);
@variable(m,x[arcs,v],Bin) #arcs covered by vehicle k
@variable(m,u[1:n+1,v])
# Each patient will be visited exactly once
@constraint(m,single1[j in patients],sum(x[(i,j),k] for i in V for k in v)==1) 
@constraint(m,single2[i in patients],sum(x[(i,j),k] for j in V for k in v)==1) 
# Drone starts from depot
@constraint(m,depot1[k in v],sum(x[(1,j),k] for j in patients)==1)
# Drone ends at depot
@constraint(m,depot2[k in v],sum(x[(i,1),k] for i in patients)==1)
# Capacity constriant
@constraint(m,capacity[k in v],sum(q[i]*x[(i,j),k] for i in patients for j in V)<=Q)
# Balance constraint
@constraint(m,balance[h in patients,k in v],sum(x[(i,h),k] for i in V)-sum(x[(h,j),k] for j in V)==0) 
# Subtour Elimination MTZ
@constraint(m,subtour1[k in v], u[1,k]==1)
@constraint(m,subtour2[k in v,i in 2:n+1], u[i,k]>=2)   
@constraint(m,subtour3[k in v,i in 2:n+1], u[i,k]<=n)
@constraint(m,subtour4[k in v,i in 2:n+1, j in 2:n+1], (u[i,k]-u[j,k]+1)<=(n-1)*(1-x[(i,j),k]))
# Maximum flying distance for each drone
@constraint(m,maxdis[k in v], sum(d[i,j]*x[(i,j),k] for i in V for j in V)<=D)                                             
# Objective Function
@objective(m,Min,5*sum(d[i,j]*x[(i,j),k] for i in V for j in V for k in v))                                                              
optimize!(m)
@show objective_value(m);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 1435 rows, 1386 columns and 11163 nonzeros
Model fingerprint: 0x429e113a
Variable types: 63 continuous, 1323 integer (1323 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [9e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 183 rows and 66 columns
Presolve time: 0.02s
Presolved: 1252 rows, 1320 columns, 10734 nonzeros
Variable types: 60 continuous, 1260 integer (1260 binary)

Root relaxation: objective 6.029585e+01, 104 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   60.29585    0   32          -   60.29585      -     -    0s
     0     0   60.41733    0   68          -   60.41733      -     - 

In [6]:
val = value.(x)
for k in v
    for i in V
        for j in V
            if val[(i,j),k]==1
                println(x[(i,j),k])
            end 
        end
    end
end

x[(1, 17),1]
x[(10, 1),1]
x[(13, 14),1]
x[(14, 10),1]
x[(17, 19),1]
x[(19, 20),1]
x[(20, 13),1]
x[(1, 18),2]
x[(2, 15),2]
x[(3, 2),2]
x[(7, 11),2]
x[(11, 21),2]
x[(15, 7),2]
x[(18, 3),2]
x[(21, 1),2]
x[(1, 6),3]
x[(4, 16),3]
x[(5, 12),3]
x[(6, 4),3]
x[(8, 9),3]
x[(9, 5),3]
x[(12, 1),3]
x[(16, 8),3]
